In [28]:
import sys
sys.path.append('/global/homes/s/sleak/.local/cori/3.6-anaconda-4.4/lib/python3.6/site-packages')
import rdflib

import logging
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)

import os
print(os.getcwd())

sys.path.append('/global/homes/s/sleak/Monitoring/Resilience/LogSet/src')
import LogsGraph    

# force re-read of LogsGraph.py in case I've edited it:
import importlib
importlib.reload(LogsGraph)

def dump(graph):
    #for s,o,p in graph:
    #    print ('{0} --- {1} --- {2}'.format(s,o,p))
    print(graph.serialize(format='n3').decode('ascii'))

import urllib.request
def parse_line_by_line(url, fmt):
    """ rdflib is really uninformative about the location of the error
        when it can't parse a file, so use this to parse line-by-line 
        and look for errors
        (hmm, does not work, fails on valid files)
    """
    f = urllib.request.urlopen(url)
    g = rdflib.ConjunctiveGraph()
    linecount=0
    for line in f.readlines():
        linecount += 1
        print("parsing line {0:d}: {1}".format(linecount,line))
        rdftext = line.strip()
        if not rdftext: continue
        try:
            g.parse(data=rdftext, format=fmt)
        except:
            print("failed to parse line {0:d} of {1}".format(linecount,url))
            print(line)
            print(g.serialize())
            raise
    print("finished parsing {0:s} successfully!".format(url))
    
import os
def old_construct(*catalog_urls, spider=False):
    graph = rdflib.ConjunctiveGraph()
    unparsed = set(catalog_urls)
    print(unparsed)
    parsed = set()
    q_peers = 'SELECT ?uri WHERE { ?cat logset:peers ?uri . }'
    # get the catalogs:
    while len(unparsed) > 0:
        for url in unparsed:
            fmt = rdflib.util.guess_format(url) 
            print("parsing {0}".format(url))
            graph.parse(url, format=fmt)
            print ("graph now has {0} statements".format(len(graph)))
            parsed.add(url)
        unparsed = set()   
        if spider:
            for peer in graph.query(q_peers):
                url = peer['uri']
                if not url in parsed:
                    unparsed.add(url)
                    
            #graph.parse(peer['uri'], format=rdflib.util.guess_format(peer['uri']))
                    
    # get the LogSets:
    q_logsets = ''' SELECT ?uri 
                    WHERE {
                        ?cat dcat:dataset ?uri . 
                    }'''
    for logset in graph.query(q_logsets):
        print("parsing found logset url: {0}".format(str(logset)))
        url = logset['uri']
        fmt = rdflib.util.guess_format(url)
        graph.parse(url, format=fmt)
        print ("graph now has {0} statements".format(len(graph)))
    return graph

# quick sanity check:
#parse_line_by_line('file:/global/u1/s/sleak/Monitoring/Resilience/LogSet/examples/nersc-entities.ttl', 'turtle')
#g1 = rdflib.ConjunctiveGraph().parse("file:/global/u1/s/sleak/Monitoring/Resilience/LogSet/examples/index.ttl", format='turtle')
#print("got this far...")
#print(g1.serialize(format='n3').decode('ascii'))

print("hello")

graph = LogsGraph.construct('nersc-catalog.ttl', spider=True)

query = '''select ?id ?title where { ?id a dcat:Catalog .
                                     ?id dct:title ?title . }'''
print("\nCatalogs:")
for row in graph.query(query):
    print('{0}: {1}'.format(str(row[0]),str(row[1])))

query = '''select ?id ?desc where { ?id a logset:LogSet .
                                     ?id dct:description ?desc . }'''
print("\nLogSets:")
for row in graph.query(query):
    print('{0}: {1}'.format(str(row[0]),str(row[1])))

    
#query = 'select ?id ?val WHERE { ?id a ?val . }'
#for row in graph.query(query):
#    print('{0} --- {1}'.format(str(row[0]),str(row[1])))
#    #print(row)
#
#
#query = '''SELECT ?id ?what ?path
#WHERE {
#?id a logset:ConcreteLog . 
#?id dcat:downloadURL ?path .
#?id logset:isInstanceOf ?what .
#} '''
#
#print("concretelogs:")
#for row in graph.query(query):
#    print(row)
#
#query = '''SELECT ?id ?path
#WHERE {
#?id a logset:ConcreteLog .
#?id logset:isInstanceOf dict:console_logfile . 
#?id dcat:downloadURL ?path .
#} '''
#print("console logfiles:")
#for row in graph.query(query):
#    print('{0} --- {1}'.format(str(row[0]),str(row[1]))) 
#dump(graph)
# why am I not finding the console-log that is in the snl index?
# ah, typo in index.ttl, point dict prefix at vocab instead .. fixed now
# first incarnation will prob be rather fragile...
                
    

#[ns for ns in graph.namespaces()]

DEBUG:root:parsed has: set()
DEBUG:root:unparsed has: {'http://portal.nersc.gov/project/mpccc/sleak/resilience/datasets/logset#', 'nersc-catalog.ttl'}
DEBUG:root:looking for http://portal.nersc.gov/project/mpccc/sleak/resilience/datasets/logset.ttl
DEBUG:root:looking for nersc-catalog.ttl
DEBUG:root:parsed has: {'http://portal.nersc.gov/project/mpccc/sleak/resilience/datasets/logset#', 'nersc-catalog.ttl'}
DEBUG:root:unparsed has: {'file:///global/u1/s/sleak/Monitoring/Resilience/LogSet/examples/nersc.ttl', 'file:///global/u1/s/sleak/Monitoring/Resilience/LogSet/examples/snl-catalog.ttl', 'file:///global/u1/s/sleak/Monitoring/Resilience/LogSet/examples/index#'}
DEBUG:root:looking for file:///global/u1/s/sleak/Monitoring/Resilience/LogSet/examples/nersc.ttl
DEBUG:root:looking for file:///global/u1/s/sleak/Monitoring/Resilience/LogSet/examples/snl-catalog.ttl
DEBUG:root:looking for file:///global/u1/s/sleak/Monitoring/Resilience/LogSet/examples/index.ttl
DEBUG:root:parsed has: {'nersc-ca

/global/u1/s/sleak/Monitoring/Resilience/LogSet/examples
hello

Catalogs:
file:///global/u1/s/sleak/Monitoring/Resilience/LogSet/examples/nersc-catalog#: Example catalog for datasets at NERSC
file:///global/u1/s/sleak/Monitoring/Resilience/LogSet/examples/snl-catalog#: Imaginary Catalog for SNL

LogSets:
file:///global/u1/s/sleak/Monitoring/Resilience/LogSet/examples/index#: a partial sample of cori log data for testing logset tools
file:///global/u1/s/sleak/Monitoring/Resilience/LogSet/examples/snl-index#: an example of a remote dataset


In [29]:
#import rdflib.tools.rdf2dot
#stream = open('test.dot', 'w')
#rdflib.tools.rdf2dot.rdf2dot(graph, stream)
#stream.close()

[ns for ns in graph.namespaces()]
#dir(rdflib.namespace)

#print(rdflib.namespace._RDFNamespace)
#print(rdflib.namespace.RDFS)
#print (rdflib.namespace.namespaces())
#, RDFS, OWL, XSD, FOAF, SKOS, DOAP, DC, DCTERMS, VOID: print (rdflib.namespace.i)

[('xml', rdflib.term.URIRef('http://www.w3.org/XML/1998/namespace')),
 ('rdf', rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#')),
 ('rdfs', rdflib.term.URIRef('http://www.w3.org/2000/01/rdf-schema#')),
 ('xsd', rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#')),
 ('dcat', rdflib.term.URIRef('http://www.w3.org/ns/dcat#')),
 ('owl', rdflib.term.URIRef('http://www.w3.org/2002/07/owl#')),
 ('dct', rdflib.term.URIRef('http://purl.org/dc/terms/')),
 ('foaf', rdflib.term.URIRef('http://xmlns.com/foaf/0.1/')),
 ('adms', rdflib.term.URIRef('http://www.w3.org/ns/adms#')),
 ('',
  rdflib.term.URIRef('http://portal.nersc.gov/project/mpccc/sleak/resilience/logset#')),
 ('logset',
  rdflib.term.URIRef('http://portal.nersc.gov/project/mpccc/sleak/resilience/datasets/logset#')),
 ('default1',
  rdflib.term.URIRef('file:///global/u1/s/sleak/Monitoring/Resilience/LogSet/examples/nersc-catalog#')),
 ('default2',
  rdflib.term.URIRef('file:///global/u1/s/sleak/Monitoring/Resilience/

In [30]:
dump(graph)
# find datasets, read them, and if they are LogSets, add them to the graph:

@prefix : <http://portal.nersc.gov/project/mpccc/sleak/resilience/logset#> .
@prefix adms: <http://www.w3.org/ns/adms#> .
@prefix cray: <file:///global/u1/s/sleak/Monitoring/Resilience/LogSet/examples/cray-dict#> .
@prefix dcat: <http://www.w3.org/ns/dcat#> .
@prefix dct: <http://purl.org/dc/terms/> .
@prefix default1: <file:///global/u1/s/sleak/Monitoring/Resilience/LogSet/examples/nersc-catalog#> .
@prefix default2: <file:///global/u1/s/sleak/Monitoring/Resilience/LogSet/examples/nersc#> .
@prefix default3: <file:///global/u1/s/sleak/Monitoring/Resilience/LogSet/examples/snl-catalog#> .
@prefix default4: <file:///global/u1/s/sleak/Monitoring/Resilience/LogSet/examples/index#> .
@prefix default5: <file:///global/u1/s/sleak/Monitoring/Resilience/LogSet/examples/snl-index#> .
@prefix default6: <file:///global/u1/s/sleak/Monitoring/Resilience/LogSet/examples/snl-entities#> .
@prefix dict: <http://portal.nersc.gov/project/mpccc/sleak/resilience/datasets/dict#> .
@prefix foaf: <http://xmln

In [27]:
for row in graph.query('''select ?id ?pred ?obj where { ?id a logset:LogSet . 
            ?id ?pred ?obj . }'''):
    #print ('{0} {1} {2}'.format(str(row[0]),str(row[1]),str(row[2])))
    #print ('{0} {1}'.format(str(row[1]),str(row[2])))
    print (str(row[0]))



file:///global/u1/s/sleak/Monitoring/Resilience/LogSet/examples/index#
file:///global/u1/s/sleak/Monitoring/Resilience/LogSet/examples/index#
file:///global/u1/s/sleak/Monitoring/Resilience/LogSet/examples/index#
file:///global/u1/s/sleak/Monitoring/Resilience/LogSet/examples/index#
file:///global/u1/s/sleak/Monitoring/Resilience/LogSet/examples/index#
file:///global/u1/s/sleak/Monitoring/Resilience/LogSet/examples/index#
file:///global/u1/s/sleak/Monitoring/Resilience/LogSet/examples/index#
file:///global/u1/s/sleak/Monitoring/Resilience/LogSet/examples/index#
file:///global/u1/s/sleak/Monitoring/Resilience/LogSet/examples/index#
file:///global/u1/s/sleak/Monitoring/Resilience/LogSet/examples/index#
file:///global/u1/s/sleak/Monitoring/Resilience/LogSet/examples/index#
file:///global/u1/s/sleak/Monitoring/Resilience/LogSet/examples/index#
file:///global/u1/s/sleak/Monitoring/Resilience/LogSet/examples/index#
file:///global/u1/s/sleak/Monitoring/Resilience/LogSet/examples/index#
file:/